In [3]:
import numpy as np
import pandas as pd

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler, SMOTE

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, classification_report

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
data = pd.read_csv('/Users/chldu/uci-secom.csv')

In [5]:
data

,Time,0,1,2,3,4,5,6,7,8,...,581,582,583,584,585,586,587,588,589,Pass/Fail
0,2008-07-19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,...,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,2008-07-19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2008-07-19 13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2008-07-19 14:43:00,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,...,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1
4,2008-07-19 15:22:00,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,...,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,2008-10-16 15:13:00,2899.41,2464.36,2179.7333,3085.3781,1.4843,100.0,82.2467,0.1248,1.3424,...,203.1720,0.4988,0.0143,0.0039,2.8669,0.0068,0.0138,0.0047,203.1720,-1
1563,2008-10-16 20:49:00,3052.31,2522.55,2198.5667,1124.6595,0.8763,100.0,98.4689,0.1205,1.4333,...,NaN,0.4975,0.0131,0.0036,2.6238,0.0068,0.0138,0.0047,203.1720,-1
1564,2008-10-17 05:26:00,2978.81,2379.78,2206.3000,1110.4967,0.8236,100.0,99.4122,0.1208,NaN,...,43.5231,0.4987,0.0153,0.0041,3.0590,0.0197,0.0086,0.0025,43.5231,-1
1565,2008-10-17 06:01:00,2894.92,2532.01,2177.0333,1183.7287,1.5726,100.0,98.7978,0.1213,1.4622,...,93.4941,0.5004,0.0178,0.0038,3.5662,0.0262,0.0245,0.0075,93.4941,-1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 592 entries, Time to Pass/Fail
dtypes: float64(590), int64(1), object(1)
memory usage: 7.1+ MB


In [11]:
data.columns[data.isna().mean() >= 0.25]

Index(['72', '73', '85', '109', '110', '111', '112', '157', '158', '220',
       '244', '245', '246', '247', '292', '293', '345', '346', '358', '382',
       '383', '384', '385', '492', '516', '517', '518', '519', '578', '579',
       '580', '581'],
      dtype='object')

# preprocessing

In [8]:
def preprocess_inputs(df):               #모델에 필요한 형식에 이미지를 적절하게 맞추기위한 것
    df = df.copy()
    
   
    df = df.drop('Time', axis=1)        #time 열 삭제
    
    
    missing_value_column = df.columns[df.isna().mean() >= 0.25]         #누락값의 평균이 25% 넘는 열 삭제
    df = df.drop(missing_value_column, axis=1)
    
   
    for column in df.columns:
        df[column] = df[column].fillna(df[column].mean())        # 평균으로 비어있는 값들 채우기
        
   
    single_value_column = ['5', '13', '42', '49', '52', '69', '97', '141', '149', '178', '179', '186', '189', '190',
        '191', '192', '193', '194', '226', '229', '230', '231', '232', '233', '234', '235', '236',
        '237', '240', '241', '242', '243', '256', '257', '258', '259', '260', '261', '262', '263',
        '264', '265', '266', '276', '284', '313', '314', '315', '322', '325', '326', '327', '328',
        '329', '330', '364', '369', '370', '371', '372', '373', '374', '375', '378', '379', '380',
        '381', '394', '395', '396', '397', '398', '399', '400', '401', '402', '403', '404', '414',
        '422', '449', '450', '451', '458', '461', '462', '463', '464', '465', '466', '481', '498',
        '501', '502', '503', '504', '505', '506', '507', '508', '509', '512', '513', '514', '515',
        '528', '529', '530', '531', '532', '533', '534', '535', '536', '537', '538']   #하나의 값만 가지는 열 제거 (함수고려)
    
    df = df.drop(single_value_column, axis=1)
    
    #give text labes to the training examples
    df['Pass/Fail'] = df['Pass/Fail'].replace({-1: "PASS", 1:"FAIL"})
    
    #split df into X and Y
    Y = df['Pass/Fail']
    X = df.drop('Pass/Fail', axis =1)
    
    # train test split
    Y = df['Pass/Fail']
    X = df.drop('Pass/Fail', axis=1)
    
    # Train-test split
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7, shuffle=True, random_state=1)
    
    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

    return X_train, X_test, Y_train, Y_test


In [9]:
X_train, X_test, Y_train, Y_test = preprocess_inputs(data)

In [10]:
X_train

,0,1,2,3,4,6,7,8,9,10,...,576,577,582,583,584,585,586,587,588,589
390,-0.349832,-0.784979,-0.702110,-0.837070,-0.062886,0.550891,0.079727,1.660382,0.575205,-2.431983,...,-0.229337,-0.101179,-1.421245,-0.104493,0.090875,-0.091661,-0.896661,-0.895327,-0.972313,-0.222332
635,0.033698,0.353357,0.015921,-1.183386,-0.051551,0.845131,-0.117085,-1.250633,0.022325,0.591865,...,-0.236675,0.059216,-0.063064,-0.367069,-0.378555,-0.351676,0.896361,0.747328,0.485680,-0.307801
78,0.246294,0.059202,-0.586681,0.429161,-0.052493,-0.356155,0.263419,-0.036013,-0.166304,-0.468738,...,-0.208856,0.044560,-1.155514,-0.185286,-0.065602,-0.176462,-0.488429,0.120526,0.054910,0.135286
733,0.315914,-1.701055,0.354622,0.405983,-0.058342,0.791001,0.027244,0.827113,3.098939,-0.107682,...,-0.214130,-0.328386,-1.037412,0.232143,0.122170,0.233756,0.952392,-0.884520,-0.773496,-0.787101
1262,-0.448795,0.573974,-0.926520,-0.573652,-0.059723,0.447072,0.355265,-1.193634,-1.285073,-1.168285,...,-0.195474,-0.249656,-1.509822,-0.037166,-0.159488,-0.028790,-0.632511,1.730761,1.811128,1.406800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,0.760898,-0.466310,0.543717,-1.204002,-0.054619,0.995094,-0.340139,-0.701000,-1.239541,0.275941,...,-0.219221,-0.656602,-1.096463,0.009963,-0.034307,0.017244,-1.152807,0.477155,0.386272,1.968328
905,-0.920769,0.379892,0.543337,-0.652297,-0.061495,-0.202822,-0.195810,0.057628,0.139406,0.501601,...,-0.213599,-0.647856,1.413220,-0.225682,-0.284669,-0.223534,-0.040173,-0.549505,-0.574678,-0.484059
1096,0.659998,-0.060836,2.263423,1.479871,-0.047401,-1.023442,-0.051481,-0.994138,0.399585,-0.615418,...,-0.216786,-0.198393,0.616026,-0.272811,-0.128193,-0.268303,-0.296319,-0.938555,-0.972313,-0.591078
235,0.423734,-0.074230,-0.101032,-0.341214,-0.062369,-0.213299,-0.222052,-0.521861,-2.176185,-0.175380,...,-0.267046,-0.597438,-1.568874,-0.205484,-0.003011,-0.191936,-2.001291,-0.787258,-0.773496,1.767280


In [11]:
Y_train

390     PASS
635     PASS
78      PASS
733     PASS
1262    PASS
        ... 
715     PASS
905     PASS
1096    PASS
235     FAIL
1061    PASS
Name: Pass/Fail, Length: 1096, dtype: object